In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
import os
import glob
from tqdm import tqdm
import matplotlib

# 백그라운드 실행 설정
matplotlib.use('Agg')

# --- [설정 영역] ---
DATA_ROOT = './data'
VIZ_ROOT = './visualize'

LINE_STYLES = {
    'Pass': '-', 'Pass_Corner': '-', 'Pass_Freekick': '-', 'Cross': '-', 'Goal Kick': '-', 'Throw-In': '-', 
    'Carry': '--', 'Take-On': '--', 
    'Shot': '-.', 'Shot_Corner': '-.', 'Shot_Freekick': '-.', 'Penalty Kick': '-.', 'Goal': '-.', 'Own Goal': '-.',
}
DEFAULT_STYLE = ':' 

# --- [헬퍼 함수] ---
def draw_pitch(ax):
    ax.add_patch(Rectangle((0, 0), 105, 68, fill=False, edgecolor='black', linewidth=2))
    ax.plot([52.5, 52.5], [0, 68], color='black', alpha=0.3, linestyle='-', linewidth=1.5)
    ax.add_patch(Circle((52.5, 34), 9.15, fill=False, edgecolor='black', alpha=0.3))
    ax.set_xlim(0, 105)
    ax.set_ylim(0, 68)
    ax.set_aspect('equal')
    ax.axis('off')

    # 공격 방향 표시
    ax.arrow(40, -5, 25, 0, head_width=2, head_length=2, fc='gray', ec='gray', alpha=0.5)
    ax.text(52.5, -10, "Attack Direction", ha='center', fontsize=8, color='gray')

def get_color(is_home):
    return 'red' if is_home else 'blue'

# --- [정적 이미지 저장 함수] ---
def save_static_image(df, save_path, title_str):
    fig, ax = plt.subplots(figsize=(8, 6))
    draw_pitch(ax)
    ax.set_title(f"{title_str}", fontsize=10)

    for i in range(len(df)):
        row = df.iloc[i]
        color = get_color(row['is_home'])
        style = LINE_STYLES.get(row['type_name'], DEFAULT_STYLE)
        
        ax.arrow(row['start_x'], row['start_y'], 
                 row['end_x'] - row['start_x'], row['end_y'] - row['start_y'],
                 head_width=1.2, head_length=1.2, fc=color, ec=color, 
                 linestyle=style, length_includes_head=True, alpha=0.8, linewidth=1.2)
        
        if i > 0:
            prev_row = df.iloc[i-1]
            if (prev_row['end_x'] != row['start_x']) or (prev_row['end_y'] != row['start_y']):
                ax.plot([prev_row['end_x'], row['start_x']], 
                        [prev_row['end_y'], row['start_y']], 
                        color='gray', linestyle=':', linewidth=1.0, alpha=0.5)

    ax.text(df.iloc[0]['start_x'], df.iloc[0]['start_y'], 'S', color='green', fontweight='bold', fontsize=12, ha='center', va='center', zorder=10)
    ax.text(df.iloc[-1]['end_x'], df.iloc[-1]['end_y'], 'E', color='black', fontweight='bold', fontsize=12, ha='center', va='center', zorder=10)

    try:
        plt.savefig(save_path, dpi=100, bbox_inches='tight')
    except Exception as e:
        print(f"Error saving image: {e}")
    finally:
        plt.close(fig)

# --- [메인 실행부] ---
def process_all():
    csv_files = glob.glob(os.path.join(DATA_ROOT, '*', '*', '*.csv'))
    print(f"총 {len(csv_files)}개의 Phase 파일을 찾았습니다. 이미지 생성을 시작합니다.")

    for file_path in tqdm(csv_files, desc="이미지 생성 중"):
        try:
            # 1. 데이터 로드
            df = pd.read_csv(file_path)
            if df.empty: continue
            
            df = df.sort_values('time_seconds').reset_index(drop=True)
            
            # 2. 경로 및 ID 파싱
            path_parts = os.path.normpath(file_path).split(os.sep)
            
            game_id = path_parts[-3]       # 126283
            episode_id = path_parts[-2]    # 1
            file_name = path_parts[-1]     # phase_1_HOME.csv
            phase_name = os.path.splitext(file_name)[0] # phase_1_HOME

            # 3. 저장 경로 설정 (단순화됨)
            # 구조: visualize/{game_id}/{episode_id}/
            target_dir = os.path.join(VIZ_ROOT, game_id, episode_id)
            os.makedirs(target_dir, exist_ok=True)

            # 4. 이미지 파일명 설정 (phase_name.png)
            # 예: visualize/126283/1/phase_1_HOME.png
            save_path = os.path.join(target_dir, f"{phase_name}.png")

            # 5. 타이틀 및 저장
            title_str = f"G{game_id} | Ep{episode_id} | {phase_name}"
            save_static_image(df, save_path, title_str)
            
        except Exception as e:
            print(f"\nError processing file {file_path}: {e}")
            continue

if __name__ == "__main__":
    if not os.path.exists(DATA_ROOT):
        print(f"Error: '{DATA_ROOT}' 폴더를 찾을 수 없습니다.")
    else:
        process_all()
        print("\n모든 이미지 저장 작업이 완료되었습니다.")

총 66731개의 Phase 파일을 찾았습니다. 이미지 생성을 시작합니다.


이미지 생성 중: 100%|██████████| 66731/66731 [2:41:35<00:00,  6.88it/s]  


모든 이미지 저장 작업이 완료되었습니다.
